In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

In [2]:
!kaggle datasets download -d Multiple Skin Disease Detection and Classification
!unzip skin-cancer-dataset.zip

usage: kaggle [-h] [-v] [-W] {competitions,c,datasets,d,kernels,k,models,m,files,f,config} ...
kaggle: error: unrecognized arguments: Disease Detection and Classification
unzip:  cannot find or open skin-cancer-dataset.zip, skin-cancer-dataset.zip.zip or skin-cancer-dataset.zip.ZIP.


In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("pritpal2873/multiple-skin-disease-detection-and-classification")

print("Path to dataset files:", path)

100%|██████████| 785M/785M [00:10<00:00, 78.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/pritpal2873/multiple-skin-disease-detection-and-classification/versions/4


In [4]:
data = '/root/.cache/kagglehub/datasets/pritpal2873/multiple-skin-disease-detection-and-classification/versions/4/Skin Cancer Dataset'

In [16]:
data_dir = data
batch_size = 32
img_size = (224, 224)

In [17]:
datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    validation_split=0.2
)

In [24]:
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset="training",
    shuffle=True
)

val_generator = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    subset="validation",
    shuffle=False
)

Found 3288 images belonging to 9 classes.
Found 819 images belonging to 9 classes.


In [22]:
base_model = tf.keras.applications.DenseNet121(
    weights='imagenet',
    include_top=False,
    input_shape=(224, 224, 3)
)
base_model.trainable = False
for layer in base_model.layers[-10:]:
    layer.trainable = True

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(9, activation='softmax')
])

In [23]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


103/103 ━━━━━━━━━━━━━━━━━━━━ 0s 634ms/step - accuracy: 0.2657 - loss: 2.1101

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()